In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Bidirectional
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import copy
from sklearn.model_selection import train_test_split
import pymysql

In [ ]:
# Connecting to the database and fetching the data for analysis
connection = pymysql.connect(host='localhost',
                             port = 3306,
                             user=input('Enter user: '),
                             password=input('Enter password: '),
                             database='scatec_data',
                             cursorclass=pymysql.cursors.DictCursor)

with connection:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "select * from inverter_data_short_v2"
        cursor.execute(sql)      
        result = cursor.fetchall()

In [ ]:
# Converting the raw data from the SQL table into a DataFrame
rawdata = pd.DataFrame(result)

In [ ]:
rawdata = rawdata.sort_values('rowid_inv')

In [ ]:
rawdata = rawdata.drop('ERR0607', axis = 1)

In [ ]:
rawdata = rawdata.fillna(value=np.nan)
rawdata = rawdata.fillna(0)
for col in rawdata.columns:
    if rawdata[col].dtype in ('object', '<M8[ns]'):
        try:
            rawdata[col] = rawdata[col].astype('float64')
        except: 
            pass
#ts = rawdata['Ts']
rowidinv = rawdata['rowid_inv']
rawdata = rawdata.select_dtypes(exclude=['object', '<M8[ns]'])
#rawdata['Ts'] = ts
rawdata['rowid_inv'] = rowidinv

In [ ]:
# Limiting data to production hour data
rawdata_prd = rawdata[(rawdata["Total_Active_Power_Measurement"]>0)]

In [ ]:
# Limiting data to data without the decided failure type
rawdata_prd = rawdata_prd[(rawdata_prd['ERR0607']<=0)]

In [ ]:
# Setting the figure size for the box-and-whiskers-plot
sns.set(rc={"figure.figsize":(16, 8)})

In [ ]:
# Plotting box-and-whiskers data for a comprehensive amount of columns
ax = sns.boxplot(data=rawdata_prd[rawdata_prd.columns[0:10]])
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

In [ ]:
# Plotting box-and-whiskers data for a comprehensive amount of columns
ax = sns.boxplot(data=rawdata_prd[rawdata_prd.columns[10:20]])
ax.set_xticklabels(['Measurement variable %s'%str(i) for  i in range(1,11)],rotation=30)

In [ ]:
# Plotting box-and-whiskers data for a comprehensive amount of columns
sns.set(font_scale = 2)
ax = sns.boxplot(data=rawdata_prd[rawdata_prd.columns[20:30]])
ax.set_xticklabels(['Variable %s'%str(i) for  i in range(1,11)],rotation=20)
plt.savefig('bwplot.png')

In [ ]:
# Plotting box-and-whiskers data for a comprehensive amount of columns
ax = sns.boxplot(data=rawdata_prd[rawdata_prd.columns[30:40]])
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

In [ ]:
# Plotting box-and-whiskers data for a comprehensive amount of columns
ax = sns.boxplot(data=rawdata_prd[rawdata_prd.columns[40:50]])
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

In [ ]:
# Plotting box-and-whiskers data for a comprehensive amount of columns
ax = sns.boxplot(data=rawdata_prd[rawdata_prd.columns[50:59]])
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

In [ ]:
# Removing outlier data based on the box-and-whiskers plots
A_1 = rawdata_prd['Section_1_DC_Current_Measurement'].quantile(0.99)
A_2 = rawdata_prd['Section_2_DC_Current_Measurement'].quantile(0.99)
T_2 = rawdata_prd['Temperature_2-AC_connections_cabinet_ambient_temperature'].quantile(0.99)
T_11 = rawdata_prd['Temperature_11-External_ambient_temperature'].quantile(0.99)
T_10 = rawdata_prd['Temperature_10-Coolant_temperature_at_the_output_of_the_power_ca'].quantile(0.99)
T_9 = rawdata_prd['Total_Active_Energy_fed_into_the_grid_by_inverter'].quantile(0.99)
P_IH = rawdata_prd['Liquid_Cooling_input_presure'].quantile(0.99)
P_OH = rawdata_prd['Liquid_Cooling_output_presure'].quantile(0.99)
P_G1 = rawdata_prd['DC_Energy_generated_from_panels_on_Section_1'].quantile(0.99)
P_G2 = rawdata_prd['DC_Energy_generated_from_panels_on_Section_2'].quantile(0.99)
P_T = rawdata_prd['Total_Active_Energy_fed_into_the_grid_by_inverter'].quantile(0.99)
P_R = rawdata_prd['Total_Capacitive_Reactive_Energy_in_the_inverter'].quantile(0.99)
T_M = rawdata_prd['ModuleTemperature'].quantile(0.98)
P_2L = rawdata_prd['Section_2_DC_Voltage_Measurement'].quantile(0.02)
P_1L = rawdata_prd['Section_1_DC_Voltage_Measurement'].quantile(0.02)
P_IL = rawdata_prd['Liquid_Cooling_input_presure'].quantile(0.02)
P_OL = rawdata_prd['Liquid_Cooling_output_presure'].quantile(0.02)
T_10_L = rawdata_prd['Temperature_10-Coolant_temperature_at_the_output_of_the_power_ca'].quantile(0.01)
T_2_L = rawdata_prd['Temperature_11-External_ambient_temperature'].quantile(0.01)
S_L = rawdata_prd['SoilingIndex'].quantile(0.02)





rawdata_prd = rawdata_prd[(rawdata_prd['Section_1_DC_Current_Measurement']<A_1)&
                         (rawdata_prd['Section_2_DC_Current_Measurement']<A_2)&
                            (rawdata_prd['Temperature_2-AC_connections_cabinet_ambient_temperature']<T_2)&
                         (rawdata_prd['Temperature_11-External_ambient_temperature']<T_11)&
                         (rawdata_prd['Temperature_10-Coolant_temperature_at_the_output_of_the_power_ca']<T_10)&
                         (rawdata_prd['Temperature_11-External_ambient_temperature']>T_2_L)&
                         (rawdata_prd['Temperature_10-Coolant_temperature_at_the_output_of_the_power_ca']>T_10_L)&
                         (rawdata_prd['Temperature_9-Coolant_temperature_at_the_input_of_the_power_cabi']<T_9)&
                         (rawdata_prd['Liquid_Cooling_input_presure']<P_IH)&
                          (rawdata_prd['Liquid_Cooling_output_presure']<P_OH)&
                          (rawdata_prd['DC_Energy_generated_from_panels_on_Section_1']<P_G1)&
                          (rawdata_prd['DC_Energy_generated_from_panels_on_Section_2']<P_G2)&
                          (rawdata_prd['Section_2_DC_Voltage_Measurement']>P_2L)&
                          (rawdata_prd['Section_1_DC_Voltage_Measurement']>P_1L)&
                          (rawdata_prd['Liquid_Cooling_input_presure']>0)&
                          (rawdata_prd['Liquid_Cooling_output_presure']>0)&
                         (rawdata_prd['Total_Active_Energy_fed_into_the_grid_by_inverter']<P_T)&
                          (rawdata_prd['Total_Capacitive_Reactive_Energy_in_the_inverter']<P_R)&
                         (rawdata_prd['ModuleTemperature']<T_M)&
                         (rawdata_prd['SoilingIndex']>S_L)]

In [ ]:
# Resetting index to cope with removed data
rawdata_prd = rawdata_prd.reset_index() 

In [ ]:
# Plotting the distribution of cycle length
df_max_rul = rawdata_prd[['Cycle_ID', 'RUL']].groupby('Cycle_ID').max()
plt.figure(frameon=False, dpi=100)
df_max_rul['RUL'].hist(bins=100, figsize=(12, 8))
plt.xlim(0)
plt.xlabel('Cycle length [Days]', fontsize  = 20)
plt.ylabel('Number of cycles', fontsize  = 20)
plt.title('Distribution of cycle length', fontsize  = 22)
plt.xticks(fontsize  = 18)
plt.yticks(fontsize  = 18)
plt.savefig('ruldist_v2.png')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    return [rmse, variance]

In [ ]:
def reCycle(dataframe, from_RUL, to_RUL, minimal_length, cycle_column = 'Cycle_ID',
            RUL_column = 'RUL', random_state = 1, min_end = 100):
    # Truncating maintenance cycles to end at a random point prior to failure
    rd = np.random.RandomState(random_state)
    newdata = pd.DataFrame()
    for cyc in np.unique(dataframe[cycle_column]):
        cycsize = len(np.unique(dataframe[(dataframe[cycle_column] == cyc)]['RUL']))
        if cycsize >= minimal_length:
            try:
                new_end = rd.randint((to_RUL+1),min(min_end,(min(cycsize,from_RUL)) - minimal_length))
            except: 
                new_end = to_RUL
            print(cyc, ' : ',new_end, ',', cycsize)
            recycled = dataframe[(dataframe[RUL_column] >= new_end)&(dataframe[cycle_column] == cyc)]
            newdata = newdata.append(recycled, ignore_index = True)
        else:
            pass
    return newdata

In [ ]:
# Removing rowid column (Only for PCA-HMM pipeline)
rawdata_prd = rawdata_prd.drop('rowid_inv', axis = 1)

In [ ]:
recycled_raw = reCycle(rawdata_prd, 3000, 1, 2,'Cycle_ID','RUL', random_state = 84)

In [ ]:
# Splitting the data into training and testing data for PCA-HMM by their Cycle-ID
trains_hmm, tests_hmm = train_test_split(np.unique(recycled_raw['Cycle_ID']), test_size = 0.3, random_state = 1)

In [ ]:
X_hmm_trn = rawdata_prd[(rawdata_prd['Cycle_ID'].isin(trains_hmm))]
y_hmm_trn = rawdata_prd[(rawdata_prd['Cycle_ID'].isin(trains_hmm))]['RUL']
X_hmm_tst = recycled_raw[(recycled_raw['Cycle_ID'].isin(tests_hmm))]
y_hmm_tst = recycled_raw[(recycled_raw['Cycle_ID'].isin(tests_hmm))]['RUL']

In [ ]:
X_hmm_trn = X_hmm_trn.reset_index()
y_hmm_trn = y_hmm_trn.reset_index()
X_hmm_tst = X_hmm_tst.reset_index()
y_hmm_tst = y_hmm_tst.reset_index()

In [ ]:
extents_trn = []
for cyc in np.unique(X_hmm_trn['Cycle_ID']):
    # Adding the locations of the training cycles in the dataset to a list
    extents_trn.append((X_hmm_trn[(X_hmm_trn['Cycle_ID'] == cyc)]['RUL'].idxmax(),X_hmm_trn[(X_hmm_trn['Cycle_ID'] == cyc)]['RUL'].idxmin()))

In [ ]:
extents_tst = []
for cyc in np.unique(X_hmm_tst['Cycle_ID']):
    # Adding the locations of the training cycles in the dataset to a list
    extents_tst.append((X_hmm_tst[(X_hmm_tst['Cycle_ID'] == cyc)]['RUL'].idxmax(),X_hmm_tst[(X_hmm_tst['Cycle_ID'] == cyc)]['RUL'].idxmin()))

In [ ]:
X_hmm_trn = X_hmm_trn.drop('RUL', axis = 1)
X_hmm_tst = X_hmm_tst.drop('RUL', axis = 1)

In [ ]:
# Performing standard scaling on the data
from sklearn.preprocessing import StandardScaler as sc
sc = sc()
X_hmm_trn = sc.fit_transform(X_hmm_trn)
X_hmm_tst = sc.transform(X_hmm_tst)


In [ ]:
# Plotting an explained variance plot for the PCA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
pca = PCA()
pca.fit(X_hmm_trn)
data_trn = pca.transform(X_hmm_trn)
data_tst = pca.transform(X_hmm_tst)


exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

fig2 = px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}, title = 'Explained variance plot for PCA'
)
fig2.update_layout(title_x=0.5)
fig2.update_layout(
    font_family="Calibri",
    font_color="black",
    title_font_family="Calibri",
    title_font_color="black",
    font_size = 20
)
fig2.show()

In [ ]:
# Performing PCA using three components
pca2 = PCA(n_components = 3)
pca2.fit(X_hmm_trn)
data_trn = pca2.transform(X_hmm_trn)
data_test = pca2.transform(X_hmm_tst)

In [ ]:
X_hmm_trn = pd.DataFrame(data_trn)
X_hmm_tst = pd.DataFrame(data_test)

In [ ]:
y_hmm_trn = pd.DataFrame(y_hmm_trn)
y_hmm_tst = pd.DataFrame(y_hmm_tst)

# Start transition matrix for HMM with 15 hidden states
transmat = np.array([[0.5,0.125,0.1,0.075,0.05,0.025,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.025],
                       [0,0.5,0.125,0.1,0.075,0.05,0.025,0.025,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.025], 
                       [0,0,0.5,0.125,0.1,0.075,0.05,0.05,0.0125,0.0125,0.0125,0.0125,0.0125,0.0125,0.025], 
                       [0,0,0,0.5,0.125,0.1,0.075,0.05,0.05,0.025,0.0125,0.0125,0.0125,0.0125,0.025], 
                       [0,0,0,0,0.5,0.125,0.1,0.075,0.05,0.05,0.025,0.025,0.0125,0.0125,0.025],
                     [0,0,0,0,0,0.5,0.125,0.1,0.075,0.05,0.025,0.025,0.025,0.025,0.05],
                       [0,0,0,0,0,0,0.5,0.125,0.1,0.075,0.05,0.05,0.025,0.025,0.05], 
                       [0,0,0,0,0,0,0,0.5,0.2,0.125,0.075,0.05,0.025,0.0125,0.0125], 
                       [0,0,0,0,0,0,0,0,0.5,0.2,0.125,0.075,0.05,0.025,0.025], 
                       [0,0,0,0,0,0,0,0,0,0.5,0.2,0.125,0.075,0.05,0.05], 
                       [0,0,0,0,0,0,0,0,0,0,0.5,0.2,0.125,0.1,0.075], 
                       [0,0,0,0,0,0,0,0,0,0,0,0.5,0.275,0.125,0.1], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0.5,0.3,0.2], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0.7,0.3], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])

# Start probability matrix for HMM with 15 hidden states
startmat = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

In [ ]:
pca_ts = PCA(n_components = 6)
pca_ts.fit(features_filtered)
data = pca_ts.transform(features_filtered)


In [ ]:
# Start transition matrix for HMM with 10 hidden states

transmat = np.array([[0.5,0.175,0.125,0.075,0.05,0.025,0.0125,0.0125,0.0125,0.0125],
                       [0,0.5,0.2,0.125,0.075,0.05,0.025,0.0125,0.0125,0], 
                       [0,0,0.5,0.2,0.125,0.075,0.05,0.025,0.0125,0.0125], 
                       [0,0,0,0.5,0.2,0.125,0.075,0.05,0.025,0.025], 
                       [0,0,0,0,0.5,0.2,0.125,0.075,0.05,0.05], 
                       [0,0,0,0,0,0.5,0.2,0.125,0.1,0.075], 
                       [0,0,0,0,0,0,0.5,0.275,0.125,0.1], 
                       [0,0,0,0,0,0,0,0.5,0.3,0.2], 
                       [0,0,0,0,0,0,0,0,0.7,0.3], 
                       [0,0,0,0,0,0,0,0,0,1]])
# Start probability matrix for HMM with 10 hidden states
startmat = np.array([1,0,0,0,0,0,0,0,0,0])



In [ ]:
# Start transition matrix for HMM with 5 hidden states
transmat = np.array([[0.6,0.15,0.1125,0.0875,0.05], 
                       [0,0.7,0.15,0.1,0.05], 
                       [0,0,0.7,0.2,0.1], 
                       [0,0,0,0.8,0.2], 
                       [0,0,0,0,1]])
# Start probability matrix for HMM with 5 hidden states
startmat = np.array([1,0,0,0,0])



In [ ]:
# Removing cycles that are too short for analysis
extents_trn = [tup for tup in extents_trn if tup[-1]>(tup[0])]
extents_tst = [tup for tup in extents_tst if tup[-1]>(tup[0])]

In [ ]:
# Training 20 different HMM models with n components and different random states
from tqdm import tqdm
from hmmlearn import hmm
results = []
ground_truth = []
results_trn = []
for RS in tqdm(range(20)):
    lr = hmm.GaussianHMM(n_components=10, covariance_type="full", n_iter = 1000, params = 'tmc', init_params = 'mc', random_state=RS)
    lr.startprob_ = startmat
    lr.transmat_ = transmat
    lr.fit(np.concatenate([X_hmm_trn.iloc[extent[0]:extent[1]] for extent in extents_trn if len(X_hmm_trn.iloc[extent[0]:extent[1]])>0]), [len(X_hmm_trn.iloc[extent[0]:extent[1]]) for extent in extents_trn if len(X_hmm_trn.iloc[extent[0]:extent[1]])>0])
    preds = []
    refs = []
    preds_trn = []
    refs_trn = []
    for split in extents_tst:
        preds.append(lr.predict(X_hmm_tst.iloc[split[0]:split[1]]))
        refs.append(y_hmm_tst.iloc[split[0]:split[1]].values)
    for split in extents_trn:
        preds_trn.append(lr.predict(X_hmm_trn.iloc[split[0]:split[1]]))
        refs_trn.append(y_hmm_trn.iloc[split[0]:split[1]].values)
    results.append(preds)
    results_trn.append(preds_trn)
    ground_truth.append(refs)

In [ ]:
# Combining the test results from the various HMM's
predictions = []
for j in range(len(results[0])):
    predictions.append([])
    for k in range(len(results[0][j])):
        pointpreds = []
        for m in range(len(results)):
            pointpreds.append(results[m][j][k])
        predictions[j].append(int(np.mean(pointpreds)))

In [ ]:
# Combining the training results from the various HMM's
predictions_trn = []
for j in range(len(results_trn[0])):
    predictions_trn.append([])
    for k in range(len(results_trn[0][j])):
        pointpreds_trn = []
        for m in range(len(results_trn)):
            pointpreds_trn.append(results_trn[m][j][k])
        predictions_trn[j].append(int(np.mean(pointpreds_trn)))

In [ ]:
# Plotting example cycle development
figure(figsize=(16, 9), dpi=80)
fig, ax = plt.subplots()
for ind, res in range(1,6):
    ind += 1
    plt.plot(refs[res],predictions[res], label = 'Maintenance cycle %s'%ind, linewidth = 2)      
plt.ylabel('Predicted hidden state', fontsize = 16)
plt.legend()
plt.xlabel('Actual days until failure', fontsize = 16)
plt.title('HMM for Inverter failure predictions with feature extraction', fontsize = 24)
ax.set_xlim(250,0)
ax.set_ylim(0,10)
#plt.savefig('HMM_Inv_10_3_diag_invax')
plt.show()        

In [ ]:
# Plotting example cycle development as a function of the RUL
import numpy as np
plotdf = pd.DataFrame()
ruls = []
states = []
reslist = [[] for num in range(11)]
for res in range(len(predictions)):
    for ind, val in enumerate(predictions[res]):
        reslist[val].append(refs[res][ind][0])
        ruls.append(refs[res][ind][0])
        states.append(val)
        
plotdf['ruls'] = ruls
plotdf['state'] = states

allstates = range(15)
sns.set(rc={"figure.figsize":(21, 16)})
sns.set_context("paper", font_scale=2.2) 

cax = sns.boxplot(x='ruls', y='state', data = plotdf, orient = 'h')
cax.set_ylabel('Predicted Hidden state', fontsize = 24)
cax.set_title('Time until failure distributed over the predicted hidden state', fontsize = 24)
cax.set_xlabel('Actual remaining useful lifetime [Days]', fontsize = 24)
cax.set_yticklabels(['State %s\n count:%d \n '%(state+1, len(reslist[state])) for state in np.unique(states)])
plt.xlim(500,0)
plt.ylim(-1,10)
fig = cax.get_figure()
fig.savefig('HMM_State_count_8_15_diag_flipped_x');

In [ ]:
# Plotting the distribution of actual RUL for each predicted state for the HMM
import numpy as np
plotdf = pd.DataFrame()
ruls = []
states = []
reslist = [[] for num in range(11)]
for res in range(len(predictions_trn)):
    for ind, val in enumerate(predictions_trn[res]):
        reslist[val].append(refs_trn[res][ind][0])
        ruls.append(refs_trn[res][ind][0])
        states.append(val)
        
plotdf['ruls'] = ruls
plotdf['state'] = states

allstates = range(15)
sns.set(rc={"figure.figsize":(21, 16)})
sns.set_context("paper", font_scale=2.2) 

#plt.xlabel('Count = %d'%len(reslist))
#plt.ylabel('RUL')
#plt.title('HMM for Inverter failure predictions with feature extraction')
cax = sns.boxplot(x='ruls', y='state', data = plotdf, orient = 'h')
cax.set_ylabel('Predicted Hidden state', fontsize = 24)
cax.set_title('Time until failure distributed over the predicted hidden state for Training data', fontsize = 24)
cax.set_xlabel('Actual remaining useful lifetime [Days]', fontsize = 24)
cax.set_yticklabels(['State %s\n count:%d \n '%(state+1, len(reslist[state])) for state in np.unique(states)])
plt.xlim(500,0)
plt.ylim(-1,10)
fig = cax.get_figure()
fig.savefig('HMM_State_count_8_15_diag_flipped_x');


In [ ]:
stats = pd.DataFrame()
stats['state'] = states
stats['RULs'] = ruls


In [ ]:
# Preparing data to Wilcoxon test
newdf = pd.DataFrame()
maxlen = 1150
times = []
states = []
for state in range(10):
    if state != 2:
        times.extend(plotdf[(plotdf['state'] == state)]['ruls'][:maxlen])
        states.extend(plotdf[(plotdf['state'] == state)]['state'][:maxlen]+1)
newdf['state'] = states
newdf['RULs'] = times

In [ ]:
from scikit_posthocs import posthoc_wilcoxon
c = posthoc_wilcoxon(newdf, p_adjust = 'bonferroni', group_col = 'state', val_col = 'RULs')

In [ ]:
for col in c.columns:    
    c[col] = c[(c[col]>0.01)][col]

In [ ]:
sns.heatmap(c, vmax = 0.1)

In [ ]:
from numpy.random import choice
def make_rul_preds_mean_mc(preds, model,n_states = 10, n_sims = 100, max_iter = 24000):
    # Runs a Monte Carlo Simulation to estimate RUL, for each state, based on the transition matrix
    mean_vals = []
    var = []
    results = []
    output = np.array(copy.copy(preds))
    transmat = model.transmat_
    for state in tqdm(range(1,(n_states+1))):
        # State = S_n. Iterates through all the states to simulate their transition to the failed, final state
        ruls = []
        for rs in range(n_sims):
            # Runs n_sims simulations of the transition from the state S_n.
            steps = 0
            cur_state = state
            rd = np.random.RandomState(rs)
            while steps < max_iter and cur_state < n_states:
                # Simulates how many steps it takes from the state S_n to the final state, while updating the current state
                # and using the probabilities from that new, current state accordningly
                cur_state = rd.choice(np.arange(1, n_states+1), 1, p=transmat[cur_state-1])[0]
                steps += 1
            ruls.append(steps)
        # takes the mean value of the hitting times for the simulations and updates the predicted states to be this mean value
        ruls = [rul/26 for rul in ruls]
        results.append(ruls)
        var.append(np.std(ruls))
        state_mean = np.mean(ruls)
        output = np.where(output == state, state_mean, output)
        mean_vals.append(state_mean)
        print(state_mean)
    return results, output

In [ ]:
trn_sims, output = make_rul_preds_mean_mc(preds, lr, n_states = 10)

In [ ]:
# Plotting the distribution of simulated RUL for each predicted state for the Monte Carlo simulation

import numpy as np
plotdf = pd.DataFrame()
ruls = []
states = []
reslist = [[] for num in range(10)]
for state_mc in range(len(mc_tst)):
    for rul in mc_tst[state_mc]:
        reslist[state_mc].append(rul)
        ruls.append(rul)
        states.append(state_mc)
        
plotdf['ruls'] = ruls
plotdf['state'] = states

allstates = range(15)
sns.set(rc={"figure.figsize":(21, 16)})
sns.set_context("paper", font_scale=2.2) 

cax = sns.boxplot(x='ruls', y='state', data = plotdf, orient = 'h')
cax.set_ylabel('Predicted Hidden state', fontsize = 24)
cax.set_title('Time until failure distributed over the predicted hidden state', fontsize = 24)
cax.set_xlabel('Actual remaining useful lifetime [Days]', fontsize = 24)
cax.set_yticklabels(['State %s\n count:%d \n '%(state+1, len(reslist[state])) for state in np.unique(states)])
plt.xlim(500,0)
plt.ylim(-1,10)
fig = cax.get_figure()
fig.savefig('HMM_MC_State_count_3_10_flipped_x');

In [ ]:
import copy
def make_rul_preds_mean(preds, preds_train, refs_train, n_states = 10):
    # Uses the distribution of RUL per hidden state in the training data to predict
    # the RUL per hidden state in the test data
    mean_vals = []
    output = np.array(copy.copy(preds))
    for state in range(1,(n_states+1)):
        ruls = []
        for ind in range(len(preds_train)):
            if preds_train[ind] == state:
                ruls.append(refs_train[ind])
        state_mean = np.mean(ruls)
        output = np.where(output == state, state_mean, output)
        mean_vals.append(state_mean) 
        print(state_mean)
    return output      


In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
preds_trn_flat = flatten(preds_trn)
refs_trn_flat = flatten(refs_trn)

In [ ]:
refs_trn_flat = flatten(refs_trn)

In [ ]:
preds_flat = flatten(preds_n)

In [ ]:
refs_flat = flatten(refs_n)


In [ ]:
mean_est = make_rul_preds_mean(preds_flat, preds_trn_flat, y_hmm_trn['RUL'].values, n_states = 10)

In [ ]:
# Plotting  results for the RUL estimation using training values
RMSEPP_rf = np.sqrt(np.mean(np.power(refs_flat-mean_est, 2)))
plt.figure(1, figsize=(16,16), frameon=False, dpi=100)
plt.plot(refs_flat, times, 'o', label="RF, RMSEPP=".format(RMSEPP_rf))
plt.plot(refs_flat, refs_flat, '--')
plt.xlabel('Reference')
plt.ylabel('Prediction')
plt.title('RF regressor for Inverter failure predictions')
plt.legend(loc='lower right')
#plt.savefig('RF_Inv')
plt.ylim(-10, 150)
plt.show()

print('MSE test: %.3f' % (
        # mean_squared_error(y_train, y_train_pred),
        mean_squared_error(refs_flat, times)))
print('RMSE test: %.3f' % (
        # np.sqrt(mean_squared_error(y_train, y_train_pred)),
        np.sqrt(mean_squared_error(refs_flat, times))))
print('R^2 test: %.3f' % (
        # r2_score(y_train, y_train_pred),
        r2_score(refs_flat, times)))

In [ ]:
def make_rul_preds_hit(preds, model, n_states = 10):
    # Predicts RUL based on the calculated theoretical hitting times for each state
    mean_vals = []
    output = np.array(copy.copy(preds))
    iters= [0 for i in range(n_states)]
    for state in range(n_states-1,0,-1):
        # Iterating beckwards through all states - starting at the second to last state and going forwards
    
        transmat = model.transmat_
        probs = transmat[state-1]
        steps = 1
        
        for ind in range(len(probs)):
            # Iterating through the probabilities of going to the other states
            
            if (ind+1) != state:
                
                # p_ij m_jA - iters is the number of steps from the potential state to state 10
                steps += probs[ind] * iters[ind]
        # Dividing the sum of p_ij m_jAp_ij by (1-p_i), 1 - probability of staying in current state
        iters[state-1] = (steps/(1 - probs[state-1]))/26
        # /26 for 26 samples per day to get RUL in days
        print(iters)
        output = np.where(output == state, steps, output)
        mean_vals.append(iters[state-1])  
    return output

In [ ]:
hit_time = make_rul_preds_hit(preds_flat, lr, n_states = 10)

In [ ]:
# Plotting results for the hitting time prediction
#RMSEPP_rf = np.sqrt(np.mean(np.power(refs_flat-hit_time, 2)))
plt.figure(1, figsize=(5, 5), frameon=False, dpi=100)
plt.plot(refs_flat, hit_time, 'o', label="RF, RMSEPP={0:.2f}".format(RMSEPP_rf))
plt.plot(refs_flat, refs_flat, '--')
plt.xlabel('Reference')
plt.ylabel('Prediction')
plt.ylim(-10, 150)
plt.title('RF regressor for Inverter failure predictions')
plt.legend(loc='lower right')
#plt.savefig('RF_Inv')
plt.show()

print('MSE test: %.3f' % (
        # mean_squared_error(y_train, y_train_pred),
        mean_squared_error(refs_flat, hit_time)))
print('RMSE test: %.3f' % (
        # np.sqrt(mean_squared_error(y_train, y_train_pred)),
        np.sqrt(mean_squared_error(refs_flat, hit_time))))
print('R^2 test: %.3f' % (
        # r2_score(y_train, y_train_pred),
        r2_score(refs_flat, hit_time)))





In [ ]:
# Preparing data for sankey plot, mapping the state transitions

source = []
target = []
value = []
for num in range(len(predictions)):
    for i in range(len(predictions[num])):
        if predictions[num][i]!=predictions[num][i-1] and i > 0 and len(predictions[num])>1:
            source.append(predictions[num][i-1])
            target.append(predictions[num][i])
            value.append(refs[num][i])

In [ ]:
# Making a Sankey plot for the HMM predicted state transitions

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = [state for state in range(1,11)],
      color = "blue"
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value,
    hovertemplate='This link has total value %{value}<extra></extra>'
  ))])

fig.update_layout(title_text="Sankey plot for state transition", font_size=20, title_x=0.5)
fig.show()

In [ ]:
# Preparing the HMM prediction data for comparison by confusion matrix

LastPoint_states_2 = []
LastPoint_RUL_2 = []
for cyc in range(len(predictions)):
    LastPoint_states_2.append(predictions[cyc][-1])
    LastPoint_RUL_2.append(refs[cyc][-1])

In [ ]:
# Data preparation for TSFRESH-RFR method

RUL = recycled_raw.drop(['RUL'],axis = 1)

In [ ]:
recycled_raw = recycled_raw.drop('RUL', axis = 1)

In [ ]:
# Removing unneccesary columns
recycled_raw = recycled_raw.drop(['ERR00', 'ERR01', 'ERR02', 'ERR03', 'ERR04', 'ERR05', 'ERR06', 
                                  'ERR07', 'ERR08', 'ERR09', 'ERR10', 'ERR11', 'ERR12', 'ERR13', 
                                  'ERR14', 'ERR15', 'Error_Word_0','Error_Word_1','Error_Word_10',
                                  'Error_Word_11','Error_Word_12','Error_Word_13','Error_Word_14',
                                  'Error_Word_15','Error_Word_2','Error_Word_3','Error_Word_4',
                                  'Error_Word_5','Error_Word_6','Error_Word_7','Error_Word_8',
                                  'Error_Word_9'], axis = 1)

In [ ]:
# Running the feature extraction with TSFRESH
from tsfresh.feature_extraction import extract_features
fresh_cycle = extract_features(recycled_raw, column_id="Cycle_ID", column_sort="rowid_inv")

In [ ]:
# Finding the ground truth for each maintenance cycle
RULs = [min(recycled_raw['RUL'][(recycled_raw['Cycle_ID']==cycle)]) for cycle in np.unique(recycled_raw['Cycle_ID'])]



In [ ]:
# Performing imputation and feature selection for the extracted data
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(X)
features_filtered = select_features(X, RULs);


In [ ]:
# Splitting the training and test data
X_train, X_test, y_train, y_test = train_test_split(features_filtered, RULs, test_size=0.3, random_state = 1)

In [ ]:
# Running hyperparameter tuning for the Random Forest regressor
results_mse = pd.DataFrame(index = range(100, 500, 100))
results_rmse = pd.DataFrame(index = range(100, 500, 100))
results_r2 = pd.DataFrame(index = range(100, 500, 100))
for mdepth in tqdm(range(5,30,5)):
    colres_mse = []
    colres_rmse = []
    colres_r2 = []
    for n_est in range(100, 500, 100):
        mser = []
        rmser = []
        r2r = []
        for rs in range(5):
            forest = RF(n_jobs = -2, max_depth = mdepth, n_estimators = n_est, random_state = rs)

            forest.fit(X_train, y_train)
            y_pred_forest = forest.predict(X_test)
            
            mser.append(mean_squared_error(y_test, y_pred_forest))
            rmser.append(np.sqrt(mean_squared_error(y_test, y_pred_forest)))
            r2r.append(r2_score(y_test, y_pred_forest))
            # y_all = np.vstack([y_test.values,y_pred])
        colres_mse.append(np.mean(mser))
        colres_rmse.append(np.mean(rmser))
        colres_r2.append(np.mean(r2r))
    results_rmse[mdepth] = colres_rmse
    results_mse[mdepth] = colres_mse
    results_r2[mdepth] = colres_r2

In [ ]:
round(results_rmse,2)

In [ ]:
round(results_mse,2)

In [ ]:
round(results_r2,3)

In [ ]:
# Running RUL predxiction using RFR
forest = RF(n_jobs = -3, max_depth = 10, n_estimators = 400)

forest.fit(X_train, y_train)
y_pred_forest= forest.predict(X_test)


RMSEPP_rf = np.sqrt(np.mean(np.power(y_test-y_pred_forest, 2)))
plt.figure(1, figsize=(10, 10), frameon=False, dpi=100)
plt.plot(y_test, y_pred_forest, 'o', label="RF, RMSEPP={0:.2f}".format(RMSEPP_rf))
plt.plot(y_test, y_test, '--')
plt.xlabel('Reference')
plt.ylabel('Prediction')
plt.xlim(0,70)
plt.ylim(0,80)
plt.title('RF regressor for Inverter failure predictions with feature extraction')
plt.legend(loc='lower right')
#plt.savefig('RF_Inv_med')
plt.show()

print('MSE test: %.3f' % (
        # mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_pred_forest)))
print('RMSE test: %.3f' % (
        # np.sqrt(mean_squared_error(y_train, y_train_pred)),
        np.sqrt(mean_squared_error(y_test, y_pred_forest))))
print('R^2 test: %.3f' % (
        # r2_score(y_train, y_train_pred),
        r2_score(y_test, y_pred_forest)))

In [ ]:
# Preparing test data for plotting

predictions_rf = [[] for num in range(250)]
ground_truth = [[] for num in range(250)]
for num in range(len(y_pred_forest)):
    predictions_rf[int(y_test[num])].append(y_pred_forest[num])
    ground_truth[int(y_test[num])].append(y_test[num])

In [ ]:
errors = [[] for num in range(250)]

In [ ]:
predictions = [p for p in predictions if str(p) != 'nan']
ground_truth = [g for g in ground_truth if str(g) != 'nan']
predictions_rf = [p for p in predictions_rf if len(p) > 0]
ground_truth = [g for g in ground_truth if len(g) > 0]
#errors = [e for e in errors if str(e) != 'nan']

In [ ]:
for num in range(len(predictions_rf)):
    errors[num] = np.std(predictions_rf[num])
    predictions_rf[num] = np.mean(predictions_rf[num])
    ground_truth[num] = np.mean(ground_truth[num])
    

In [ ]:
# Plotting Test results for TSFRESH + RFR with error bars
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(layout = go.Layout(
title = 'Predictions of remaining useful lifetime, with error bars showing mean error',
yaxis = dict(title = 'Predicted RUL [Days]'),
xaxis = dict(title = 'Actual RUL [Days]'),
                 width=1000, height=800),  
                data=go.Scatter(
        x=ground_truth,
        y=predictions_rf,
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=errors,
            visible=True), name = 'Prediction mean values'
    ))
fig.update_layout(title_text='Predictions of RUL, with error bars showing standard deviation', title_x=0.5)
fig.update_layout(
    font_family="Calibri",
    font_color="black",
    title_font_family="Calibri",
    title_font_color="black",
    font_size = 22
)
fig.update_layout(
    legend_font_size = 20
)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.1,
    xanchor="right",
    x=0.99
))
fig.add_trace(go.Scatter(x=y_test, y=y_pred_forest, mode = "markers", name = 'Predictions'))
fig.show()


In [ ]:
# Finding and displaying feature importances by the RFR
imps = forest.feature_importances_

importances = {}
for num, impo in enumerate(imps):
    if impo > 0:
        print(X_train.columns[num], impo)
        importances[X_train.columns[num]] = impo

importances = sorted(importances.items(), key=lambda x: x[1], reverse=True)

df_imp = pd.DataFrame(importances)

In [ ]:
df_imp.head(10)

In [ ]:
def risk_classes(pred):
    # Translates RUL into a risk class for ground truth and predicted RUL
    res = copy.copy(pred)
    res = np.where(res<=5,0,res)
    res = np.where(np.logical_and(res>5, res<=20),1,res)
    res = np.where(res>20,2,res)
    return res

In [ ]:
def risk_classes_hmm(pred):
    # Translates predicted hidden state infor a risk class
    res = copy.copy(pred)
    res = np.where(res<=4,2,res)
    res = np.where(np.logical_and(res>4, res<=7),1,res)
    res = np.where(res>7,0,res)
    return res

In [ ]:
forest_classes = risk_classes(y_pred_forest)

In [ ]:
gt_classes_rf = risk_classes(y_test)

In [ ]:
hmm_classes_n = risk_classes_hmm(np.array(LastPoint_states_2))

In [ ]:
gt_classes_hmm_n = risk_classes(np.array(LastPoint_RUL_2))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_rf = f1_score(gt_classes_rf, forest_classes, average = 'macro')
print('f1-score for TSFRESH-RFR method: ', f1_rf)

f1_hmm = f1_score(gt_classes_hmm, hmm_classes, average = 'micro')
print('f1-score for PCA HMM method: ',f1_hmm)

In [ ]:
# Creating and plotting confusion matrices for the two methods
from sklearn.metrics import confusion_matrix
hmm_confmat = confusion_matrix(gt_classes_hmm_n, hmm_classes_n)
rf_confmat = confusion_matrix(gt_classes_rf, forest_classes)
sns.set(font_scale=3)

In [ ]:
a = sns.heatmap(rf_confmat, annot = True,yticklabels = range(1,4),xticklabels = range(1,4))
a.set_xlabel('Predicted class')
a.set_ylabel('True class')
a.set_title('Confusion matrix for TSFRESH Random Forest Regressor method')
plt.savefig('RF_Confplot')

In [ ]:
b = sns.heatmap(hmm_confmat, annot = True,yticklabels = range(1,4),xticklabels = range(1,4))
b.set_xlabel('Predicted class')
b.set_ylabel('True class')
b.set_title('Confusion matrix for PCA HMM method')
plt.savefig('HMM_confmat')
